# Janssen & Rollins ABM

This project constitutes an attempt to replicate the ABM from the paper "Evolution of cooperation in asymmetric commons dilemmas" by Marco A. Janssen and Nathan D. Rollins (2011). 
See http://www.sciencedirect.com/science/article/pii/S0167268111002599 for the paper. 

This version is only preliminary and still at an early stage. The code has not yet been cleaned up completely and still contains some unnecessary variables and functions. This will be gradually improved during the next weeks.

## Basic Structure

This replication model relies on the Project Mesa Framework (see github.com/projectmesa).
To get an understanding of how an ABM is built with mesa, it is useful to start with the basic blocks: the Agent and Model classes. These are imported from the mesa module.

In [ ]:
from mesa import Model, Agent

At the most basic level, an agent is initialized with a unique agent ID, and a variable where some information can be stored; as an example, each agent might have a score that is increased as the model runs.

In [3]:
class Resource_Agent(Agent):
    
     def __init__(self, unique_id):
        self.unique_id = unique_id
        self.score = 0

The Model class will later create a number of agents. To do this, both the number of agents and a function to create them will be needed:

In [4]:
class Resource_Model(Model):
    
    
    def __init__(self, N):
        self.num_agents = N
        self.create_agents()
        
    def create_agents(self):
        for i in range(self.num_agents):
            a = Resource_Agent(i, alpha, beta, eta)
            self.schedule.add(a)

The above already is sufficient to create a working model, although the created agents will not do anything yet. A model with 5 agents can be created and run as follows:

In [ ]:
test_model = Resource_Model(5)
test_model.create_agents()

## Adding a Scheduler

In order to have the agents perform a certain action in each step of the model, a step function will have to be added to the Agent class. As an example, the agents will just increase their own score by 1 each round.

In the Model class, it will have to be defined in what order the agents are activated, and how the model behaves at each step. Mesa includes several schedules, the simplest being the BaseScheduler - the agents are activated in the order they were created each round.

In [ ]:
from mesa import Model, Agent
from mesa.time import BaseScheduler

class Resource_Agent(Agent):
    
    def __init__(self, unique_id):
        self.unique_id = unique_id
        self.score = 0
        
    def step(model):
        self.score += 1

class Resource_Model(Model):
    
    def __init__(self, N):
        self.num_agents = N
        self.schedule = BaseScheduler
        self.create_agents()
        
    def create_agents(self):
        for i in range(self.num_agents):
            a = Resource_Agent(i, alpha, beta, eta)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()

    def run_model(self, steps):
        for i in range(steps):
            self.step()

This model can now be run, with 5 agents increasing their score each round, which will be repeated for 10 rounds.

In [ ]:
test_model = Resource_Model(5)
test_model.create_agents()
test_model.run(10)

## Fitting the Model to Janssen & Rollins'

In order to recreate the desired model, Agent and Model class will have to be modified to fit Janssen&Rollins Public Resource Game. The model's structure with two sub-steps per round will require changes to the Scheduler class as well.
In this first approach, the algorithm by which agents respond to each other will be omitted, and a contribution and share takeout level will be chosen at random.

### Agent Class

In the game's first round, each agent is endowed with a certain number of tokens (10), some of which can be contributed to the common public resource (CPR). Depending on the amount of contributions, this resource will be multiplied by some factor. In the second round, the agents take turns taking out some share of the CPR - the first agent gets to pick first, and so forth, until the last agent. Each step, contribution and takeout share will be chosen at random. The agents store the tokens they acquired during the game in a score variable.

In [ ]:
import random
from mesa import Model, Agent

class Resource_Agent(Agent):

    def __init__(self, unique_id):
        self.unique_id = unique_id
        self.score = 0
        self.endowment = 10
        self.contribution = random.choice(0, 10, 1)
        self.takeout_share = random.choice(0, 1, 0.1)
        
    def first_round(self, model):
        self.endowment = 10
        self.contribution = random.choice(0, 10, 1)
        self.score += (self.endowment - self.contribution)
        
    def second_round(self, model):
        self.takeout_share = random.choice(0, 1, 0.1)
        takeout = round(model.resource * self.takeout_share)
        self.score += takeout
        model.resource -= takeout

### Model Class

The model class will determine the amount of tokens currently in the CPR, create a number of agents, and run for a number of steps. Janssen&Rollins describe an S-shaped function for the generated resource: more than one contributing agent is necessary to create a CPR. Mesa features a class for data colletion - this will be used to visualize the model's outcome in the end.

In [ ]:
from mesa import DataCollector

class Resource_Model(Model):
    
    def __init__(self, N):
        self.num_agents = N
        self.schedule = BaseScheduler(self)
        self.create_agents()
        self.resource = 0
        #creating a variable to determine the amount of CPR generated in this round:
        self.sum_of_contributions = 0
        #data collector
        ar = {"Score": lambda ar: ar.score}
        self.dc = DataCollector(agent_reporters = ar)
        
    def create_agents(self):            
        for i in range(self.num_agents):
            a = Resource_Agent(i, alpha, beta, eta)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()
        #collecting data
        self.dc.collect()
        #resetting the sum of contributions needed to determine the amount of CPR:
        self.sum_of_contributions = 0
        
    def run_model(self, steps):
        for i in range(steps):
            self.step()
            
    def created_resource(self):
        if self.contribution_sum >= 0 and self.contribution_sum <= 10:
            produced_resource = 0
        elif self.contribution_sum >10 and self.contribution_sum <= 15:
            produced_resource = 5
        elif self.contribution_sum >15 and self.contribution_sum <= 20:
            produced_resource = 20
        elif self.contribution_sum >20 and self.contribution_sum <= 25:
            produced_resource = 40
        elif self.contribution_sum >25 and self.contribution_sum <= 30:
            produced_resource = 60
        elif self.contribution_sum >30 and self.contribution_sum <= 35:
            produced_resource = 75
        elif self.contribution_sum >35 and self.contribution_sum <= 40:
            produced_resource = 85
        elif self.contribution_sum >40 and self.contribution_sum <= 45:
            produced_resource = 95
        elif self.contribution_sum >45 and self.contribution_sum <= 50:
            produced_resource = 100
        return produced_resource

### Scheduler

Due to the Model's design with two rounds in each step, the scheduler class needs to be modified. Also, after each first round, the model's created_resource() function will be called to determine the amount of CPR generated.

In [ ]:
class BaseScheduler(object):
    model = None
    steps = 0
    time = 0
    agents = []

    def __init__(self, model):
        self.model = model
        self.steps = 0
        self.time = 0
        self.agents = []

    def add(self, agent):
        self.agents.append(agent)

    def remove(self, agent):
        while agent in self.agents:
            self.agents.remove(agent)

    def step(self):
    #Modified part.
        for agent in self.agents:
            agent.first_round(self.model)
        self.model.resource = self.model.created_resource()
        for agent in self.agents:            
            agent.second_round(self.model)
        self.steps += 1
        self.time += 1

    def get_agent_count(self):
        return len(self.agents)

### Running the Model

test_model = Resource_Model(5)
test_model.create_agents()
test_model.run(10)

### Visualization of Results

Since the data collector collects data on each agents score, the scores of the five agents can be shown in a table using matplotlib.

In [1]:
import matplotlib.pyplot as plt

data = test_model.dc.get_agent_vars_dataframe()
last_step_data = data[data.Step == (runs -1 )]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("Scores")
ax.set_ylabel("Score")
ax.set_xlabel("Agent")
ax.bar(5, last_step_data.Score, 0.3)

NameError: name 'test_model' is not defined